In [ ]:
from typing import Annotated, TypedDict, List, Tuple, Dict, Union, Optional, Any
from langgraph.graph import StateGraph, END
# from langchain.core.runnable import Runnable
from langgraph.graph.message import add_messages
from langchain.agents.agent import AgentExecutor
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_community.chat_models import ChatOpenAI
import os, pandas as pd
from dotenv import load_dotenv
from generate_summary import GenerateFileSummary, DataFrameSummaryGenerator
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate



# Load .env values
load_dotenv()

# --- Define the Graph State Schema ---
class State(TypedDict):
    messages: Annotated[List, add_messages]

# --- Main Class ---
class DocQA:
    """
    A class to handle document-based question answering using LangChain and Pandas.
    """

    def __init__(self):
        
        self.llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"))
        self.graph = None
        self.files_info: Dict[str, Dict[str,Any]] = {}
        self.dataframes: Dict[str, pd.DataFrame] = {}
        self.df_agents: Dict[str, AgentExecutor] = {}
        self.files_knowledge_base: str = "These are the files in the knowledge base: \n\n"
        self.all_files_summary: str = "# Uploaded Files Summary\n\n"

        # Build the state graph
        self.graph = self._build_graph()


    def load_document(self, file_paths: str, file_types: str, file_names: str)->Dict[str, Any]:
        """
        Load a document into the agent.
        Args:
            file_path (str): Path to the document.
            file_type (str): Type of the document ('xlsx' or 'csv').
            file_name (str): Name of the document.
        Returns:
            dict: Status of the loading process.
        """
        
        try:
            for file_path, file_type, file_name in zip(file_paths, file_types, file_names):
                file_summary = None
                df = None
                agent = None
                if file_type in ["xlsx", "csv"]:
                    if file_type == "xlsx":
                        df = pd.read_excel(file_path)
                    elif file_type == "csv":
                        df = pd.read_csv(file_path)
                    else:
                        raise ValueError("Unsupported file type. Please use 'xlsx' or 'csv'.")
                    print("\n\n 1 -------------------------------")
                    agent = create_pandas_dataframe_agent(
                        llm=self.llm,
                        df=df,
                        verbose=True,
                        handle_parsing_errors=True,
                        max_iterations=3,
                        allow_dangerous_code=True
                    )
                    print("\n\n 2 -------------------------------")
                    df_summary_generator:GenerateFileSummary = DataFrameSummaryGenerator(llm = self.llm)

                    summary, questions, markdown_summary = df_summary_generator.generate_summary(
                        df=df, file_name=file_name, file_format=file_type
                    )
                    print("\n\n 3 -------------------------------")
                    self.files_info[file_name] = {
                        "file_type": file_type,
                        "summary": summary,
                        "questions": questions,
                        "markdown_summary": markdown_summary,
                    }
                    self.dataframes[file_name] = df
                    self.df_agents[file_name] = agent
                    print("\n\n 4 -------------------------------")
                    file_summary = markdown_summary
                    

                    self.files_knowledge_base += f"File Name: {file_name}\nFile Type: {file_type}\nFile Content Description:{summary}\n\n"

                elif file_type == "pdf":
                    pass
                else:
                    raise ValueError("Unsupported file type. Please use 'xlsx', 'csv', or 'pdf'.")
                
                self.all_files_summary = self.all_files_summary + markdown_summary + "\n\n"
                

            
            
        except Exception as e:
            print(f"Error loading document: {e}")
            return {"status":"exception", "message": f"Exception in DocQa, load_document().\n Exception: \n{str(e)}"}
        
        return {"status":"success", "file_summary": self.all_files_summary}

    def decide_df_agent(self, question:str) -> Optional[str]:
        """
        Decide which DataFrame agent to use based on the question.
        Args:
            question (str): The user's question.
        Returns:
            str: The appropriate DataFrame file name or None if no suitable file is found.
        """
        # 1. Define the response schema fields
        response_schemas = [
            ResponseSchema(name="file_name", description="Name of file which can contain the expected answer", type="string"),
        ]

        # 2. Create the parser
        output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

        # 3. Get format instructions
        format_instructions = output_parser.get_format_instructions()

        prompt_template = """
        
        {files_knowledge_base}

        Your Task is to decide which file could contains the answer to the question and return the file name.

        Question: {question}

        Return your answer in **format** as shown below:
        {format_instructions}

        Other Instructions:
        - If you are not sure, return "None".
        - If you think that the question is not related to any file, return "None".
        - If you think that the question is related to multiple files, return the file name which is most likely to contain the answer.
        - Stick to the response format and do not return any additional information.
        """

        prompt = ChatPromptTemplate.from_template(prompt_template)

        chain = prompt | self.llm | output_parser

        # 7. Call the chain
        result = chain.invoke({"files_knowledge_base": self.files_knowledge_base, "question":question,"format_instructions" : format_instructions})

        print("\n\n Result:", result, "Type:", type(result))

        return result["file_name"] if "file_name" in result and result["file_name"] != "None" else None


    # Define the node logic
    def get_answer_from_df(self, state: State) -> State:
        """
        Get answer from the DataFrame based on the user's question.
        Args:
            state (State): The current state of the graph.
        Returns:
            State: The updated state with the answer.
        """

        question = state["messages"][-1].content

        cnt = 3

        file_name = None

        while cnt > 0:

            file_name = self.decide_df_agent(question)
            if file_name is not None and file_name in self.df_agents and file_name != "None":
                break
            cnt-= 1


        if file_name is None or file_name == "None":
            response = {"output": "Despite of multiple attempts, I am not sure which file contains the answer to your question, please try specifying the file name."}
            print(response, type(response))
            print("\n\n\n")
        else:
            state["messages"].append({"role": "assistant", "content": f"Using file: {file_name} to answer your question:{question}."})
            agent = self.df_agents.get(file_name)
            response = agent.invoke({question})
            print(response, type(response))
            print("\n\n\n")
        
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": str(response["output"])}]
        }
    
    def _build_graph(self):
        """
        Build the state graph for the question-answering process.
        Returns:
            StateGraph: The compiled state graph.
        """

        # Create a graph builder
        graph_builder = StateGraph(State)

        # Build graph
        graph_builder.add_node("get_answer_from_df", self.get_answer_from_df)
        graph_builder.set_entry_point("get_answer_from_df")
        graph_builder.add_edge("get_answer_from_df", END)

        return graph_builder.compile()

    def ask(self, question: str) -> Union[str, dict]:
        """
        Ask a question to the agent and get the answer.
        Args:
            question (str): The question to ask.
        Returns:
            str: The answer from the agent.
        """

        # Construct initial state
        state = {
            "messages": [{"role": "user", "content": question}]
        }
        # Run the graph and return final message
        result = self.graph.invoke(state)
        return str(result["messages"][-1].content)

qa_agent = None

if __name__ == "__main__":

    # df = pd.read_csv("sales_forecast.csv")
    # df_forcast : pd.DataFrame = pd.read_excel('G:/company_sales_genai/company_sales_data_qna_using_genai/data/forcast.xlsx')
    qa_agent = DocQA()
    print("DocQA agent initialized successfully.\n\n")
    file_paths = [
        'G:/company_sales_genai/company_sales_data_qna_using_genai/data/emp_data.csv',
        'G:/company_sales_genai/company_sales_data_qna_using_genai/data/forcast.xlsx',
        'G:/company_sales_genai/company_sales_data_qna_using_genai/data/wine.csv'
    ]
    file_types = ['csv', 'xlsx', 'csv']
    file_names = ['emp_data.csv', 'forcast.xlsx', 'wine.csv']
    x = qa_agent.load_document(file_paths=file_paths, file_types=file_types, file_names=file_names)
    # Define the file path (you can change the name or path as needed)
    file_path = "./output_summary.txt"
    summ = str(x.get("file_summary", "No summary available."))
    # Write the text to the file
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(summ)

    # Return the file path (optional)
    print(f"Text written to: {file_path}")


DocQA agent initialized successfully.




 1 -------------------------------


 2 -------------------------------


C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(




 Result: {'summary': 'The DataFrame contains information about employees, including their EmployeeID, Name, and DateOfJoining. There are 20 entries in total.', 'questions': ['1. What is the average length of time employees have been with the company based on the DateOfJoining column?', '2. Are there any trends in the hiring patterns based on the DateOfJoining column?', '3. What is the distribution of employee names in terms of commonality?', '4. Is there a correlation between EmployeeID and DateOfJoining?', '5. How does the distribution of DateOfJoining vary across different departments or teams?', '6. Are there any outliers in terms of the length of employment?', '7. What is the most common starting month for new employees based on DateOfJoining?', '8. Are there any patterns in the EmployeeID assignment?', '9. How has the employee base grown over time based on the DateOfJoining?', '10. Can we predict future hiring trends based on the current data?']} Type: <class 'dict'>


 3 ------

C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(




 Result: {'summary': 'The DataFrame contains data related to the sales and inventory of various products from a brand called HISENSE. It includes columns such as brand, product, product description, status, and monthly sales and inventory data. The products are categorized as air purifiers, portable ACs, and split ACs.', 'questions': ['What is the average monthly sales quantity across all products?', 'What is the distribution of sales status (HAND, PURCHASE, SALES) across different products?', 'How do sales quantities vary month over month for each product category?', 'What is the trend in sales for each product category over the months?', 'Are there any seasonal patterns in sales for specific products?', 'What is the overall inventory trend for the brand HISENSE?', 'Which product has the highest average sales quantity per month?', 'Is there any correlation between the sales of one product category and another?', 'Can we predict future sales based on historical data using time series

C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(




 Result: {'summary': 'The DataFrame contains data related to wines, including various attributes such as alcohol content, malic acid levels, color intensity, and more. There are 178 entries with 14 columns, mostly consisting of numerical values.', 'questions': ['What is the distribution of alcohol content among the different types of wine?', 'Is there a correlation between color intensity and proline content?', 'What is the average ash content for each type of wine?', 'Are there any outliers in the phenols column? How do they affect the overall analysis?', 'How does malic acid content vary based on alcohol levels?', 'What is the relationship between flavanoids and nonflavanoid phenols?', 'Can we predict wine type based on the given attributes?', 'What are the key factors influencing the quality of wine?', 'Is there a seasonal pattern in wine production based on alcohol content?', 'What clustering algorithms can be applied to segment the wines based on their characteristics?']} Type: 

In [10]:
print(qa_agent.files_knowledge_base)

These are the files in the knowledge base: 

File Name: forcast.xlsx
File Type: xlsx
File Content Description:This DataFrame contains data related to a company's products, including information such as brand, product description, product type, status, and monthly sales data from October to December of the previous year and January to June of the current year. The data also includes initial on-hand quantities and purchase orders. The products in the DataFrame are related to air conditioning units.

File Name: wine.csv
File Type: csv
File Content Description:The DataFrame contains information about wines, including various chemical properties such as alcohol content, acidity levels, phenols, color intensity, and more. There are 178 entries and 14 columns in the DataFrame.

File Name: emp_data.csv
File Type: csv
File Content Description:The DataFrame contains information about employees, including their EmployeeID, Name, and DateOfJoining. There are 20 entries in the DataFrame.




In [11]:
print(qa_agent.dataframes.keys())
print(qa_agent.df_agents.keys())

dict_keys(['forcast.xlsx', 'wine.csv', 'emp_data.csv'])
dict_keys(['forcast.xlsx', 'wine.csv', 'emp_data.csv'])


In [12]:
print(qa_agent.ask("how many employees are there in the company?"))



 Result: {'file_name': 'emp_data.csv'} Type: <class 'dict'>


> Entering new AgentExecutor chain...
Thought: To find out how many employees are there in the company, I need to count the number of rows in the dataframe. 
Action: python_repl_ast
Action Input: len(df)20The number 20 is not the count of employees, it is the number of rows in the dataframe. Let me correct this.
Action: python_repl_ast
Action Input: df['EmployeeID'].nunique()20The unique count of EmployeeIDs is 20, so there are 20 employees in the company.
Final Answer: There are 20 employees in the company.

> Finished chain.
{'input': {'how many employees are there in the company?'}, 'output': 'There are 20 employees in the company.'} <class 'dict'>




There are 20 employees in the company.
